In [ ]:
#! pip install transformers

In [ ]:
#!pip install torchtext

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import IMDB
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset

import spacy

import numpy as np

import random
import math
import time

from itertools import chain, islice

from tqdm import tqdm

import json

SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

! python -m spacy download en

spacy_en = spacy.load('en')

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>', 
            lower = True)

# should be 0 if the sentence is natural and 1 if this is encoded
LABEL = LabelField(dtype = torch.float)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [4]:
#take original dataset for first words
train_data, test_data = IMDB.splits(SRC, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


Generating data

In [5]:
data_size = 1000  #size of generated dataset

file_names = ["FLC_16.json", "FLC_32.json", "FLC_64.json", 
              "VLC_16.json", "VLC_32.json", "VLC_64.json",
              "FLC_2.json", "FLC_4.json", "FLC_8.json", 
              "VLC_2.json", "VLC_4.json", "VLC_8.json"]

In [6]:
#writing the record in json file 
def write_json(data, filename='data.json'): 
    with open(filename,'a') as f: 
        f.write(json.dumps(data)+"\n")

In [ ]:
dir_path = "./data/"

In [ ]:
#adding original texts
for text in tqdm(islice(train_data.text, 0, data_size)):
    for name in file_names:
        write_json({"text": " ".join(text), "label":1}, dir_path + "attacker_data" + name)

In [ ]:
#adding generated texts to json files
for name in file_names:
    with open("../data/texts/" + name) as json_file:
        data = json.load(json_file)

    for text in data:
        write_json({"text": text, "label":0},
                 dir_path + "attacker_data" + name)

checking for decoding and encoding validity of a model

In [ ]:
seq = list(np.random.binomial(1, 0.5, 50))

text = generate(model, 'Hello', seq, tokenizer)

In [ ]:
text

"Hello I know the first one of the game's characters in-game"

In [ ]:
_tokens = tokenizer.encode(text, return_tensors='pt')
decoded_seq = decode(model, _tokens)

In [ ]:
np.array_equal(decoded_seq, seq)